In [1]:
import os
os.chdir('../')
!pwd

/Users/shrey/Desktop/github/human_pose_estimation


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnEstimation.constants import *
from cnnEstimation.utils.common import read_yaml, create_dir


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config 

In [4]:
import os
import urllib.request as request
import zipfile
from cnnEstimation import logger
from cnnEstimation.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_zip_file(self):
        """
        Extracts the zip file to the data directory (unzip_dir).
        Function returns None.
        """
        unzip_path = self.config.unzip_dir
        zip_file_path = self.config.local_data_file

        if not os.path.exists(zip_file_path):
            raise FileNotFoundError(f'Zip file not found at {zip_file_path}')

        os.makedirs(unzip_path, exist_ok=True)

        logger.info(f'Extracting {zip_file_path} to {unzip_path}')

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f'Extraction complete: {zip_file_path} to {unzip_path}') 

In [5]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f'Error in data ingestion: {str(e)}')
    raise e

[2025-01-31 10:58:50,257 - INFO - common - YAML file loaded successfully: config/config.yaml]
[2025-01-31 10:58:50,258 - INFO - common - YAML file loaded successfully: params.yaml]
[2025-01-31 10:58:50,260 - INFO - common - Directory created successfully: artifacts]
[2025-01-31 10:58:50,265 - INFO - common - Directory created successfully: artifacts/data_ingestion]
[2025-01-31 10:58:50,267 - INFO - 1952817005 - Extracting data.zip to artifacts/data_ingestion]
[2025-01-31 10:58:50,907 - INFO - 1952817005 - Extraction complete: data.zip to artifacts/data_ingestion]
